# 패키지 불러오기

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import gc

# 중극장 데이터 불러오기

In [2]:
df = pd.read_parquet("data/Medium_Theater.parquet")

## 년도_월, 년도_월_일 컬럼 만들기

In [3]:
# 년도_월, 년도_월_일 컬럼
df["년도_월"] = df["공연일시"]
df["년도_월"] = df["년도_월"].apply(lambda x : x[0:4]+x[5:7])

df["년도_월_일"] = df["공연일시"]
df["년도_월_일"] = df["년도_월_일"].apply(lambda x : x[0:4]+x[5:7]+x[8:10])

In [4]:
df = df.astype({'년도_월':'int'})
df = df.astype({'년도_월_일':'int'})

## 년도, 월, 일, 요일 컬럼 만들기

In [5]:
df["공연일시"] = pd.to_datetime(df["공연일시"])
df["예매/취소일시"] = pd.to_datetime(df["예매/취소일시"])

In [6]:
df["년도"] = df["공연일시"].dt.year
df["월"] = df["공연일시"].dt.month
df["일"] = df["공연일시"].dt.day
df["요일"] = df["공연일시"].dt.day_name()

## 공연일시보다 예매일시가 늦은 경우 삭제

In [7]:
df[df["공연일시"] < df["예매/취소일시"]][["공연일시", "예매/취소일시"]]

,공연일시,예매/취소일시
99,2019-07-06 14:00:00,2019-07-06 14:09:00
101,2019-07-06 14:00:00,2019-07-06 14:09:00
329,2019-07-06 17:00:00,2019-07-10 17:44:00
330,2019-07-06 17:00:00,2019-07-10 17:44:00
331,2019-07-06 17:00:00,2019-07-10 17:44:00
...,...,...
4754897,2022-12-31 21:00:00,2022-12-31 21:33:00
4754898,2022-12-31 21:00:00,2022-12-31 21:33:00
4754899,2022-12-31 21:00:00,2022-12-31 21:32:00
4754900,2022-12-31 21:00:00,2022-12-31 21:32:00


In [8]:
idx = df[df["공연일시"] < df["예매/취소일시"]][["공연일시", "예매/취소일시"]].index
len(idx)

89883

In [9]:
df = df.drop(idx, axis=0)
df = df.reset_index(drop=True)

## 중복데이터 삭제

In [10]:
temp = df[df.duplicated(['입장권고유번호', '전송사업자명'], keep=False)].sort_values(by = ['입장권고유번호', '예매/취소일시'])
gb_temp = temp.groupby("입장권고유번호").sum()["예매/취소구분"]
gb_temp

입장권고유번호
0000782754        2
0000782756        2
0000782807        2
0000782809        2
0000782811        2
                 ..
W2298902870002    3
W2299434270001    3
W2299434270002    3
W2299635419001    3
W2299635419002    3
Name: 예매/취소구분, Length: 831550, dtype: int64

In [11]:
# 가장 뒤에 데이터만 살리고 앞의 데이터 모두 지우기
df = df.sort_values(by = ['공연일시', '예매/취소일시']).reset_index(drop=True)
df = df.drop_duplicates(['입장권고유번호', '전송사업자명'], keep='last')
df

,공연시설코드,전송사업자명,공연코드,공연지역명,주소,좌석수,공연일시,예매/취소일시,입장권고유번호,예매/취소구분,...,장당금액,장르명,세부장르명,좌석등급,년도_월,년도_월_일,년도,월,일,요일
0,FC222305,인터파크,PF372107,경상도,대구광역시 달서구 문화회관길 160 (장기동),455,2019-07-01 15:00:00,2019-06-28 23:22:00,T0882670981,1,...,0,뮤지컬,뮤지컬,전석무료(0),201907,20190701,2019,7,1,Monday
1,FC222305,인터파크,PF372107,경상도,대구광역시 달서구 문화회관길 160 (장기동),455,2019-07-01 15:00:00,2019-06-28 23:22:00,T0882670982,1,...,0,뮤지컬,뮤지컬,전석무료(0),201907,20190701,2019,7,1,Monday
2,FC222305,인터파크,PF372107,경상도,대구광역시 달서구 문화회관길 160 (장기동),455,2019-07-01 15:00:00,2019-06-28 23:22:00,T0882670983,1,...,0,뮤지컬,뮤지컬,전석무료(0),201907,20190701,2019,7,1,Monday
3,FC222305,인터파크,PF372107,경상도,대구광역시 달서구 문화회관길 160 (장기동),455,2019-07-01 15:00:00,2019-06-28 23:22:00,T0882670984,1,...,0,뮤지컬,뮤지컬,전석무료(0),201907,20190701,2019,7,1,Monday
4,FC222305,인터파크,PF372107,경상도,대구광역시 달서구 문화회관길 160 (장기동),455,2019-07-01 15:00:00,2019-06-28 23:22:00,T0882670985,1,...,0,뮤지컬,뮤지컬,전석무료(0),201907,20190701,2019,7,1,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4666011,FC222768,영천시민회관,PF425870,경상도,경상북도 영천시 시청로 17 (문외동),799,2022-12-31 21:50:00,2022-12-30 17:59:00,473341,1,...,5000,대중음악,대중가요,전석(10000),202212,20221231,2022,12,31,Saturday
4666012,FC222768,영천시민회관,PF425870,경상도,경상북도 영천시 시청로 17 (문외동),799,2022-12-31 21:50:00,2022-12-30 22:48:00,473348,1,...,5000,대중음악,대중가요,전석(10000),202212,20221231,2022,12,31,Saturday
4666013,FC222768,영천시민회관,PF425870,경상도,경상북도 영천시 시청로 17 (문외동),799,2022-12-31 21:50:00,2022-12-30 22:48:00,473350,1,...,5000,대중음악,대중가요,전석(10000),202212,20221231,2022,12,31,Saturday
4666014,FC222768,영천시민회관,PF425870,경상도,경상북도 영천시 시청로 17 (문외동),799,2022-12-31 21:50:00,2022-12-30 22:48:00,473351,1,...,5000,대중음악,대중가요,전석(10000),202212,20221231,2022,12,31,Saturday


## 입장권고유번호 기준으로 예매건수보다 취소건수가 많은 경우 삭제

In [12]:
# 입장권 고유번호를 기준으로 예매/취소 건수를 카운트
df["예매건수"] = ""
df["취소건수"] = ""

def TicketingCount(x):
    if x == 1:
        return 1
    else:
        return 0
    
def CancleCount(x):
    if x == 2:
        return 1
    else:
        return 0
    
df["예매건수"] = df["예매/취소구분"].apply(lambda x : TicketingCount(x))
df["취소건수"] = df["예매/취소구분"].apply(lambda x : CancleCount(x))

In [13]:
temp = df.groupby(["입장권고유번호", "전송사업자명"]).sum()[["예매건수", "취소건수"]]
len(temp)

3794063

In [14]:
# 예매건수보다 취소건수가 많은 데이터 제거하기
temp[temp["예매건수"] < temp["취소건수"]]

,,예매건수,취소건수
입장권고유번호,전송사업자명,,
0003025700RDPT,티몬,0,1
0012930486,문화N티켓,0,1
0012930508,문화N티켓,0,1
0012930584,문화N티켓,0,1
0013030597,문화N티켓,0,1
...,...,...,...
W2299635419002,위메프,0,1
W2299759870001,국립아시아문화전당,0,1
W2299815157001,국립아시아문화전당,0,1


In [15]:
idx = temp[temp["예매건수"] < temp["취소건수"]].index
len(idx)

840719

In [16]:
df["입장권고유번호제거"] = df["입장권고유번호"].apply(lambda x : 0 if x in idx else x)
idx = df[df["입장권고유번호제거"]==0].index

In [17]:
df = df.drop(idx, axis=0)
df = df.reset_index(drop=True)

In [18]:
df = df.drop("입장권고유번호제거", axis=1)

## 공연시설코드는 같은데, 주소가 다른경우 하나로 수정

In [19]:
code = df["공연시설코드"].unique()
code

array(['FC222305', 'FC222397', 'FC222759', 'FC222691', 'FC223203',
       'FC223304', 'FC222630', 'FC222389', 'FC222318', 'FC223890',
       'FC222739', 'FC222426', 'FC222298', 'FC222054', 'FC222382',
       'FC223391', 'FC222593', 'FC222049', 'FC223390', 'FC222058',
       'FC222677', 'FC223320', 'FC222898', 'FC222669', 'FC222642',
       'FC222076', 'FC223307', 'FC223835', 'FC222570', 'FC222768',
       'FC223637', 'FC222422', 'FC222637', 'FC222390', 'FC222661',
       'FC222048', 'FC223518', 'FC222099', 'FC222542', 'FC222080',
       'FC223485', 'FC223055', 'FC222629', 'FC222760', 'FC222912',
       'FC223012', 'FC223492', 'FC222463', 'FC222984', 'FC223510',
       'FC223505', 'FC222983', 'FC223418', 'FC222203', 'FC223470',
       'FC223561', 'FC223478', 'FC222415', 'FC223807', 'FC222096',
       'FC222979', 'FC223464', 'FC222455', 'FC222587', 'FC223584',
       'FC222280', 'FC222613', 'FC222628', 'FC223628', 'FC223194',
       'FC222663', 'FC222597', 'FC222615', 'FC222091', 'FC2235

In [20]:
# 공연시설코드값을 돌리면서 주소의 고유값을 리스트로 리스트에 담기
def AdressCheck(df, code):
    adress = []
    for i in code:
        ad = df[df["공연시설코드"] == i]["주소"].unique()
        adress.append(list(ad))
    return adress
adress = AdressCheck(df, code)

In [21]:
len(adress)

222

In [22]:
for i in range(len(adress)):
    if len(adress[i]) >= 2:
        print(i)

## 무료공연 유료공연 구분

In [23]:
# 전석무료(0), 전석(0), A석(0), 현장판매(0), A석 전석소외계층초청(0), '전석무료(0), 전석무료(0)'
df["좌석등급"].unique()

array(['전석무료(0)', '전석(20000)', '전석(10000)', 'R석(30000), S석(20000)', None,
       '전석(12000)', '전석(5000)', 'VIP석(40000), R석(30000), A석(20000)',
       '전석(30000)', 'R석(70000), S석(50000), A석(30000)', '전석(40000)',
       '전석(35000)', 'R석(55000)', '전석(25000)', 'VIP석(5000), 일반석(4000)',
       'R석(15000), S석(10000)', 'R석(40000), S석(30000), A석(20000)',
       '전석(77000)', 'R석(20000), S석(10000)', '일반석(25000), 사이드석(10000)',
       'R석(50000), S석(30000), A석(20000)', '전석(15000)',
       '1층석(10000), 2층석(5000)', 'R석(6000), S석(5000), A석(4000)',
       'R석(50000), S석(30000)', '1층석(15000), 2층석(11000)', '전석(50000)',
       'R석(20000), S석(15000)', 'J석(40000), R석(35000), S석(25000)',
       'R석(66000), S석(44000)', '1층 R석(30000), 2층 S석(24000)', '전석(24000)',
       'R석(30000), S석(20000), A석(10000)',
       'R석(50000), S석(30000), A석(20000), B석(10000)',
       '비지정R석(50000), 비지정S석(30000), 비지정A석(20000), 비지정B석(10000)',
       'R석(110000), S석(99000), A석(88000)', 'A석(10000), C석(5000)',
       '전석(3000)', '1층석(70

In [24]:
# 없음
df["할인종류명(관리시스템)"].unique()

array(['기타', '자체할인', '신용카드사', '다중할인'], dtype=object)

In [25]:
# 초대권, 초대, 유료회원할인 100%, 무료 공연 할인 100%, , 송년음악회 할인 0원

# 초대권 100%, 전석초대(신나는오케스트라) 0원, 송년음악회 할인 0원, 무료 공연 할인 100%, 유료회원할인 100%
# 유료회원 할인 100%, 초대, 초대권, 직원초대, 초대 100%, 패키지 할인권 100%, 무료공연 할인 100%
# 수험생 무료 공연 관람 할인 100%
df["할인종류명(전송처)"].unique()

array([None, '0', '기타', '할인정보 없음', '부산북구주민 50%', '자체할인', '전주한옥마을숙소이용객',
       '청소년할인(24세이하)', '신용카드사', '65세이상어르신', '조기예매 40%', '50%할인', '75%할인',
       '80%할인', '장애인(1~3급,동반1인) 20%', '장애인(4~6급,본인만) 20%', '없음',
       '국민생활기초생활 수급자 50%', '현대일렉트릭스앤에너지시스템', '현대중공업직원 우대', '문예회원 우대',
       '현대미포조선직원 우대', '할인없음', '현대중공업직원 가족 우대', '복지회관직원 우대', '현대중공업 임원 우대',
       '조기예매 50%', '2.0.1.9. 이벤트', '예매시 할인', '예매시 할인 19000원',
       '조기예매할인 40%', '2.0.1.9 중 주민번호 3자리 이상 있을 시 55%', '특별할인권소지자 60%',
       '기간특가할인', '현대로보틱스', '현대중공업협력사직원 우대', '전북은행카드할인', '장애우(1~3급)',
       '임직원할인', '다자녀 사랑카드 소지 가구원 20%', '가족패키지(3인이상)', '문화가 있는 날 50%',
       '예매시 할인 11000원', '경주시민할인', '[어린이공연 국악동요극] 조기 예매 할인',
       '24세 이하 청소년 및 대학생', '예술인패스 소지자', '장애인 및 동반 1인', '문화누리카드 소지자',
       '병역이행명문자 및 동반 1인', '국가유공자 및 동반 1인', '경로우대자(만 65세 이상) 및 동반 1인',
       '전라북도도민', '예매시 40%', '2.0.1.9 중 주민번호 3자리 이상 있을 시',
       '울산광역시장 발급 자원봉사증 소지자 20%',
       '장애인복지법 등록 장애인 50% ※ 장애인등록증 소지자 (1~3급은 동반 1명 포함 / 4~6급은 본인만)',
       '복지할

In [26]:
df["결제수단명(관리시스템)"].unique()

array(['기타', '카드', '무통장', '현금', '다중결제', '상품권'], dtype=object)

In [27]:
# 무료결제, "0원 결제"
df["결제수단명(전송처)"].unique()

array(['현금', None, '무통장 입금', '신용카드', '카카오페이(카카오머니)', '카드', '기타', '무통장',
       '무통장입금', '신용카드 수기결제', '신용카드 간편결제', '네이버예약 카드', '실시간계좌이체',
       '카카오페이(카드)', '휴대폰결제', '네이버예약 계좌이체', '네이버예약 포인트', '이지웰포인트',
       'Onepay(카드)', '다중결제', '네이버페이 포인트', 'PAYCO 포인트', '2교대 복리 포인트',
       '스마일페이(카드)', '휴대폰', 'Onepay(계좌)', '국민카드', '농협은행,포인트,OK캐쉬백', '해피머니',
       'PAYCO', '해피머니+신용카드', '국민은행', '비씨카드', 'S머니 + 신용카드', '컬쳐캐쉬',
       '휴대폰 간편결제', '현대카드', '신용카드,하나투어 마일리지', '신용카드,네이버 2천원 할인쿠폰',
       '신용카드,네이버 2천원 할인쿠폰,하나투어 마일리지', '하나투어 마일리지,휴대폰', '네이버 2천원 할인쿠폰,휴대폰',
       '네이버 2천원 할인쿠폰,무통장 입금', '네이버 2천원 할인쿠폰,무통장 입금,하나투어 마일리지', '신용카드,휴대폰',
       '신용카드,카카오페이_5천원권_201901', '꿈꾸는모바일 포인트', '이지웰포인트+신용카드',
       '온라인, 마일리지결제', '급여이체 현중', '외상거래처', '신용카드, 신용카드', '외환은행', '신한카드',
       '신한은행', '하나[구 하나SK]', '롯데카드', '신용카드, 이용권', '이제너두', '농협은행', '삼성카드',
       '시티카드', '신한카드,OK캐쉬백', '농협카드', '무료', '하나은행', '비씨카드,포인트,OK캐쉬백',
       '비씨카드,포인트', '99', '가스공사포인트', '새마을금고',
       '신용카드,네이버 2천원 할인쿠폰,하나투어 마일리지,L포인트', '기업은행

In [28]:
contains_lst = [
    "전석무료(0)", "전석(0)", "A석(0)", "현장판매(0)", "A석 전석소외계층초청(0)", "전석무료(0), 전석무료(0)",
    "초대권 100%", "전석초대(신나는오케스트라) 0원", "송년음악회 할인 0원", "무료 공연 할인 100%", "유료회원할인 100%",
    "유료회원 할인 100%", "초대", "초대권", "직원초대", "초대 100%", "패키지 할인권 100%", "무료공연 할인 100%",
    "수험생 무료 공연 관람 할인 100%",
    "무료결제", "0원 결제"
]
temp = df[(df["좌석등급"].apply(lambda x : x in contains_lst))|(df["할인종류명(전송처)"].apply(lambda x : x in contains_lst))|(df["결제수단명(전송처)"].apply(lambda x : x in contains_lst))]
temp[["예매/취소금액", "할인금액", "장당금액", "좌석등급", "할인종류명(전송처)", "결제수단명(전송처)"]]

,예매/취소금액,할인금액,장당금액,좌석등급,할인종류명(전송처),결제수단명(전송처)
0,0,0,0,전석무료(0),None,현금
1,0,0,0,전석무료(0),None,현금
2,0,0,0,전석무료(0),None,현금
3,0,0,0,전석무료(0),None,현금
4,0,0,0,전석무료(0),None,현금
...,...,...,...,...,...,...
2853011,0,0,0,전석무료(0),None,현금
2853012,0,0,0,전석무료(0),None,현금
2853040,0,0,0,전석무료(0),None,현금
2853041,0,0,0,전석무료(0),None,현금


In [29]:
# 여기서 무료공연인데 예매/취소금액이 0이 아닌 비정상 데이터 수정
temp[temp["예매/취소금액"]!=0][["예매/취소금액", "할인금액", "장당금액", "좌석등급", "할인종류명(전송처)", "결제수단명(전송처)"]]

,예매/취소금액,할인금액,장당금액,좌석등급,할인종류명(전송처),결제수단명(전송처)
91095,153000,0,153000,전석무료(0),None,카드
91096,153000,0,153000,전석무료(0),None,카드
135946,1000,0,1000,현장판매(0),None,현금
135947,1000,0,1000,현장판매(0),None,현금
135948,1000,0,1000,현장판매(0),None,카드
...,...,...,...,...,...,...
1910466,1000,0,1000,전석무료(0),None,무통장
2086098,10000,0,10000,전석무료(0),None,카드
2086099,10000,0,10000,전석무료(0),None,카드
2086100,10000,0,10000,전석무료(0),None,카드


In [30]:
# 예매금액과 할인금액을 뒤바꿔주기
idx = temp[temp["예매/취소금액"]!=0][["예매/취소금액", "할인금액", "장당금액", "좌석등급", "할인종류명(전송처)", "결제수단명(전송처)"]].index

for i in idx:
    df.loc[i, "할인금액"] = df.loc[i, "예매/취소금액"]
    df.loc[i, "예매/취소금액"] = 0

In [31]:
df.iloc[idx][["예매/취소금액", "할인금액", "장당금액", "좌석등급", "할인종류명(전송처)", "결제수단명(전송처)"]].head(3)

,예매/취소금액,할인금액,장당금액,좌석등급,할인종류명(전송처),결제수단명(전송처)
91095,0,153000,153000,전석무료(0),None,카드
91096,0,153000,153000,전석무료(0),None,카드
135946,0,1000,1000,현장판매(0),None,현금


In [32]:
# 무료여부 붙여주기
idx = temp.index
len(idx)

389663

In [33]:
# 무료를 1로 유료를 2로
df["무료여부"] = ""

for i in idx:
    df.loc[i, "무료여부"] = 1

df = df.replace({"무료여부": {"" : 2}})

# 유료공연의 가격수정

## Case 1.

In [34]:
# case 1. 예매금액 제외 0원
# 예매금액과 장당금액을 맞추어주는 작업 시행.
pay_df = df[df["무료여부"]==2][["공연코드", "예매/취소금액", "할인금액", "장당금액", "할인종류명(전송처)", "좌석등급", "결제수단명(전송처)"]]
case_one = pay_df[(pay_df["예매/취소금액"]!=0)&(pay_df["할인금액"]==0)&(pay_df["장당금액"]==0)]
case_one

,공연코드,예매/취소금액,할인금액,장당금액,할인종류명(전송처),좌석등급,결제수단명(전송처)
580,PF361976,10000,0,0,None,전석(20000),무통장 입금
581,PF361976,10000,0,0,None,전석(20000),무통장 입금
582,PF361976,20000,0,0,None,전석(20000),신용카드
584,PF361976,20000,0,0,None,전석(20000),신용카드
585,PF361976,20000,0,0,None,전석(20000),신용카드
...,...,...,...,...,...,...,...
836429,PF389263,10000,0,0,None,전석(20000),S머니 + 신용카드
836430,PF389263,10000,0,0,None,전석(20000),S머니 + 신용카드
836431,PF389263,10000,0,0,None,전석(20000),S머니 + 신용카드
836432,PF389263,10000,0,0,None,전석(20000),신용카드


In [35]:
idx = case_one.index

for i in idx:
    df.loc[i, "장당금액"] = df.loc[i, "예매/취소금액"]

In [36]:
df.iloc[idx][["공연코드", "예매/취소금액", "할인금액", "장당금액"]]

,공연코드,예매/취소금액,할인금액,장당금액
580,PF361976,10000,0,10000
581,PF361976,10000,0,10000
582,PF361976,20000,0,20000
584,PF361976,20000,0,20000
585,PF361976,20000,0,20000
...,...,...,...,...
836429,PF389263,10000,0,10000
836430,PF389263,10000,0,10000
836431,PF389263,10000,0,10000
836432,PF389263,10000,0,10000


## Case 2.

In [37]:
# case 3. 장당금액 제외 0원
pay_df = df[df["무료여부"]==2][["공연코드", "예매/취소금액", "할인금액", "장당금액", "할인종류명(전송처)", "좌석등급", "결제수단명(전송처)"]]
case_two = pay_df[(pay_df["예매/취소금액"]==0)&(pay_df["할인금액"]==0)&(pay_df["장당금액"]!=0)]
case_two

,공연코드,예매/취소금액,할인금액,장당금액,할인종류명(전송처),좌석등급,결제수단명(전송처)
34841,PF373734,0,0,8000,None,전석(10000),현금
34925,PF373734,0,0,10000,None,전석(10000),현금
34978,PF373734,0,0,7000,None,전석(10000),현금
34979,PF373734,0,0,7000,None,전석(10000),현금
60783,PF373013,0,0,7000,None,전석(10000),현금
...,...,...,...,...,...,...,...
2613227,PF424367,0,0,1000,None,전석(1000),현금
2625410,PF423529,0,0,4000,None,전석(5000),현금
2625411,PF423529,0,0,4000,None,전석(5000),현금
2666084,PF425688,0,0,8000,None,전석(10000),현금


In [38]:
# 장당금액과 예매금액을 맞추어주는 작업 시행.
idx = case_two.index

for i in idx:
    df.loc[i, "예매/취소금액"] = df.loc[i, "장당금액"]

## Case 3.

In [39]:
# case 3. 예매금액만 0원
# 장당금액 - 할인금액 = 예매금액 맞춰주기
# 할인금액이 장당금액보다 큰 애들은 제거.
pay_df = df[df["무료여부"]==2][["공연코드", "예매/취소금액", "할인금액", "장당금액", "예매/취소구분", "좌석등급", "할인종류명(전송처)", "결제수단명(전송처)"]]
case_thr = pay_df[(pay_df["예매/취소금액"]==0)&(pay_df["할인금액"]!=0)&(pay_df["장당금액"]!=0)]
case_thr

,공연코드,예매/취소금액,할인금액,장당금액,예매/취소구분,좌석등급,할인종류명(전송처),결제수단명(전송처)
8595,PF360569,0,30000,30000,1,"R석(30000), S석(20000)",기타,기타
8596,PF360569,0,30000,30000,1,"R석(30000), S석(20000)",기타,기타
8597,PF360569,0,30000,30000,1,"R석(30000), S석(20000)",기타,기타
8598,PF360569,0,30000,30000,1,"R석(30000), S석(20000)",기타,기타
8599,PF360569,0,30000,30000,1,"R석(30000), S석(20000)",기타,기타
...,...,...,...,...,...,...,...,...
2850646,PF425620,0,40000,10000,1,전석(10000),None,99
2850701,PF425620,0,30000,10000,1,전석(10000),None,99
2850778,PF425620,0,20000,10000,1,전석(10000),None,99
2850817,PF425620,0,80000,10000,1,전석(10000),None,99


In [40]:
# 할인금액이 장당금액보다 작은 정상적인 경우
case_thr[case_thr["할인금액"] < case_thr["장당금액"]]

,공연코드,예매/취소금액,할인금액,장당금액,예매/취소구분,좌석등급,할인종류명(전송처),결제수단명(전송처)


In [41]:
# 위의 경우가 없기 떄문에 같지 않은 애들 제거.
new_case_thr = case_thr[case_thr["할인금액"] != case_thr["장당금액"]]

In [42]:
idx = new_case_thr.index
df = df.drop(idx, axis=0)
df = df.reset_index(drop=True)

In [43]:
# 정상적인 경우는 예매금액 = 장당금액 - 할인금액으로 맞춰주기

In [44]:
# df 바뀌었으므로 다시 생성
pay_df = df[df["무료여부"]==2][["공연코드", "예매/취소금액", "할인금액", "장당금액", "예매/취소구분", "좌석등급", "할인종류명(전송처)", "결제수단명(전송처)"]]
case_thr = pay_df[(pay_df["예매/취소금액"]==0)&(pay_df["할인금액"]!=0)&(pay_df["장당금액"]!=0)]
case_thr

,공연코드,예매/취소금액,할인금액,장당금액,예매/취소구분,좌석등급,할인종류명(전송처),결제수단명(전송처)
8595,PF360569,0,30000,30000,1,"R석(30000), S석(20000)",기타,기타
8596,PF360569,0,30000,30000,1,"R석(30000), S석(20000)",기타,기타
8597,PF360569,0,30000,30000,1,"R석(30000), S석(20000)",기타,기타
8598,PF360569,0,30000,30000,1,"R석(30000), S석(20000)",기타,기타
8599,PF360569,0,30000,30000,1,"R석(30000), S석(20000)",기타,기타
...,...,...,...,...,...,...,...,...
2806761,PF422663,0,35000,35000,1,"R석(35000), S석(30000)",기타,현금
2806782,PF422663,0,35000,35000,1,"R석(35000), S석(30000)",기타,현금
2806790,PF425517,0,15000,15000,1,전석(15000),자체할인,기타
2806791,PF425517,0,15000,15000,1,전석(15000),자체할인,기타


In [45]:
idx = case_thr.index

for i in idx:
    df.loc[i, "예매/취소금액"] = df.loc[i, "장당금액"] - df.loc[i, "할인금액"]

## case 4.

In [46]:
# 같은 공연장의 같은공연을 본 사람들의 평균 금액을 이용
df["공연시설_공연_코드"] = df["공연시설코드"] + df["공연코드"]

In [47]:
# case 4. 모두 0인 경우
pay_df = df[df["무료여부"]==2][["공연시설_공연_코드", "예매/취소금액", "할인금액", "장당금액"]]
pay_df[(pay_df["예매/취소금액"]==0)&(pay_df["할인금액"]==0)&(pay_df["장당금액"]==0)].head(3)

,공연시설_공연_코드,예매/취소금액,할인금액,장당금액
601,FC222397PF361976,0,0,0
602,FC222397PF361976,0,0,0
603,FC222397PF361976,0,0,0


In [48]:
# 3개 컬럼이 모두 누락된 애들을 위해서 공연시설_공연_코드를 이용
# 단수차이가 있기 때문에 예매/취소금액, 할인금액만 그대로 쓰고 장당금액은 두개의 합계로 계산
temp = df[df["무료여부"]==2].groupby("공연시설_공연_코드").mean().reset_index()[["공연시설_공연_코드", "예매/취소금액", "할인금액", "장당금액"]]
temp = round(temp)
temp = temp.astype({'예매/취소금액':'int64'})
temp = temp.astype({'할인금액':'int64'})
temp = temp.astype({'장당금액':'int64'})
temp.head()

,공연시설_공연_코드,예매/취소금액,할인금액,장당금액
0,FC222006PF310578,4702,0,4702
1,FC222006PF317488,7778,0,7778
2,FC222007PF301466,19209,0,19209
3,FC222007PF303033,24430,0,24430
4,FC222007PF303808,30038,852,30877


In [49]:
# 근데, 유료공연이고 공연을 본 모든 사람들의 평균 금액 3개가 모두 0인 데이터 제거
temp[(temp["예매/취소금액"]==0)&(temp["할인금액"]==0)&(temp["장당금액"]==0)]

,공연시설_공연_코드,예매/취소금액,할인금액,장당금액
56,FC222048PF301332,0,0,0
60,FC222048PF305630,0,0,0
61,FC222048PF306638,0,0,0
63,FC222048PF307315,0,0,0
65,FC222048PF311890,0,0,0
...,...,...,...,...
6922,FC223975PF319419,0,0,0
6927,FC224024PF398557,0,0,0
6984,FC224848PF300338,0,0,0
7047,FC224991PF423705,0,0,0


In [50]:
idx = temp[(temp["예매/취소금액"]==0)&(temp["할인금액"]==0)&(temp["장당금액"]==0)]["공연시설_공연_코드"].unique()
new_idx = []
for i in range(len(idx)):
    temp_idx = df[df.loc[:,"공연시설_공연_코드"] == idx[i]].index
    new_idx.append(temp_idx)

In [51]:
res = []

for i in range(len(new_idx)):
    num = len(new_idx[i])
    res.append(num)

In [52]:
res_idx = []

for i in range(len(new_idx)):
    for j in range(len(new_idx[i])):
        num = new_idx[i][j]
        res_idx.append(num)

In [53]:
df = df.drop(res_idx, axis=0)
df = df.reset_index(drop=True)

In [54]:
# 데이터가 바뀌었으므로 temp다시 생성
temp = df[df["무료여부"]==2].groupby("공연시설_공연_코드").mean().reset_index()[["공연시설_공연_코드", "예매/취소금액", "할인금액", "장당금액"]]
temp = round(temp)
temp = temp.astype({'예매/취소금액':'int64'})
temp = temp.astype({'할인금액':'int64'})
temp = temp.astype({'장당금액':'int64'})
temp.head()

,공연시설_공연_코드,예매/취소금액,할인금액,장당금액
0,FC222006PF310578,4702,0,4702
1,FC222006PF317488,7778,0,7778
2,FC222007PF301466,19209,0,19209
3,FC222007PF303033,24430,0,24430
4,FC222007PF303808,30038,852,30877


In [55]:
temp[(temp["예매/취소금액"]==0)&(temp["할인금액"]==0)&(temp["장당금액"]==0)]

,공연시설_공연_코드,예매/취소금액,할인금액,장당금액


In [56]:
pay_df = df[df["무료여부"]==2][["공연시설_공연_코드", "예매/취소금액", "할인금액", "장당금액", "예매/취소구분"]]
case_four = pay_df[(pay_df["예매/취소금액"]==0)&(pay_df["할인금액"]==0)&(pay_df["장당금액"]==0)]
case_four.head()

,공연시설_공연_코드,예매/취소금액,할인금액,장당금액,예매/취소구분
601,FC222397PF361976,0,0,0,1
602,FC222397PF361976,0,0,0,1
603,FC222397PF361976,0,0,0,1
604,FC222397PF361976,0,0,0,1
605,FC222397PF361976,0,0,0,1


In [57]:
# 평균값을 df에 반영하여 수정
idx = case_four["공연시설_공연_코드"].index
len(idx)

328317

In [58]:
code = case_four["공연시설_공연_코드"].unique()
len(code)

2333

In [59]:
case_four_temp = temp[temp["공연시설_공연_코드"].apply(lambda x : x in code)]

In [60]:
def MakePrice(df, idx, temp):
    for i in idx:
        code = df.iloc[i]["공연시설_공연_코드"]
        df.loc[i, "예매/취소금액"] = temp[temp["공연시설_공연_코드"]==code]["예매/취소금액"].values[0]
        df.loc[i, "할인금액"] = temp[temp["공연시설_공연_코드"]==code]["할인금액"].values[0]
        df.loc[i, "장당금액"] = df.loc[i, "예매/취소금액"] + df.loc[i, "할인금액"]
    return df

In [61]:
df = MakePrice(df, idx, case_four_temp)

In [62]:
df.iloc[idx][["공연시설_공연_코드", "예매/취소금액", "할인금액", "장당금액", "예매/취소구분"]]

,공연시설_공연_코드,예매/취소금액,할인금액,장당금액,예매/취소구분
601,FC222397PF361976,7528,0,7528,1
602,FC222397PF361976,7528,0,7528,1
603,FC222397PF361976,7528,0,7528,1
604,FC222397PF361976,7528,0,7528,1
605,FC222397PF361976,7528,0,7528,1
...,...,...,...,...,...
2785630,FC222759PF426633,15642,0,15642,1
2785631,FC222759PF426633,15642,0,15642,1
2785638,FC222759PF426633,15642,0,15642,1
2785641,FC222759PF426633,15642,0,15642,1


# 이상치 처리

In [63]:
df.describe()

,좌석수,예매/취소구분,예매/취소금액,예매/취소방식코드,결제수단코드,할인금액,할인종류코드,장당금액,년도_월,년도_월_일,년도,월,일,예매건수,취소건수,무료여부
count,2.786276e+06,2786276.0,2.786276e+06,2.786276e+06,2.786276e+06,2.786276e+06,2.786276e+06,2.786276e+06,2.786276e+06,2.786276e+06,2.786276e+06,2.786276e+06,2.786276e+06,2786276.0,2786276.0,2.786276e+06
mean,6.582555e+02,1.0,1.629438e+04,2.970634e+01,2.670760e+01,8.720989e+02,9.326983e+01,1.697630e+04,2.021040e+05,2.021042e+07,2.020955e+03,8.525250e+00,1.683404e+01,1.0,0.0,1.860761e+00
std,2.064196e+02,0.0,3.921952e+04,4.379182e+01,4.230045e+01,4.817126e+03,2.274344e+01,3.764735e+04,1.201727e+02,1.201717e+04,1.208046e+00,2.954547e+00,8.716693e+00,0.0,0.0,3.461953e-01
min,3.000000e+02,1.0,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,2.019070e+05,2.019070e+07,2.019000e+03,1.000000e+00,1.000000e+00,1.0,0.0,1.000000e+00
25%,4.780000e+02,1.0,5.000000e+03,2.000000e+00,2.000000e+00,0.000000e+00,9.900000e+01,5.000000e+03,2.020060e+05,2.020063e+07,2.020000e+03,7.000000e+00,9.000000e+00,1.0,0.0,2.000000e+00
50%,6.860000e+02,1.0,1.050000e+04,2.000000e+00,2.000000e+00,0.000000e+00,9.900000e+01,1.200000e+04,2.021120e+05,2.021122e+07,2.021000e+03,9.000000e+00,1.700000e+01,1.0,0.0,2.000000e+00
75%,8.240000e+02,1.0,2.100000e+04,9.900000e+01,9.900000e+01,0.000000e+00,9.900000e+01,2.270300e+04,2.022080e+05,2.022082e+07,2.022000e+03,1.100000e+01,2.400000e+01,1.0,0.0,2.000000e+00
max,9.990000e+02,1.0,5.260000e+07,9.900000e+01,9.900000e+01,7.000000e+05,9.900000e+01,5.260000e+07,2.022120e+05,2.022123e+07,2.022000e+03,1.200000e+01,3.100000e+01,1.0,0.0,2.000000e+00


## 예매/취소금액

In [64]:
# 예매/취소금액
5.260000e+07

52600000.0

In [65]:
# 예매/취소금액이 5260만원인 경우 -> 정상으로 봄.
# 좌석수가 962석이고 결제수단명이 외상거래처(기업) 이므로. 통대관한 경우로 보임.
# 시기도 코로나 이후로 적절함.
df[df["예매/취소금액"]==52600000][["전송사업자명", "주소", "좌석수", "공연일시", "예매/취소일시", "입장권고유번호", "예매/취소금액", "할인금액", "장당금액", "좌석등급", "할인종류명(전송처)", "결제수단명(전송처)"]]

,전송사업자명,주소,좌석수,공연일시,예매/취소일시,입장권고유번호,예매/취소금액,할인금액,장당금액,좌석등급,할인종류명(전송처),결제수단명(전송처)
2373637,현대예술관,울산광역시 동구 명덕로 10 (서부동),962,2022-10-26 10:00:00,2022-10-25 17:31:00,2022102501020202001010,52600000,0,52600000,"1층석(35000), 2층석(25000)",할인없음,외상거래처


In [66]:
df[df["예매/취소금액"]>200000][["전송사업자명", "좌석수", "공연일시", "예매/취소일시", "입장권고유번호", "예매/취소금액", "할인금액", "장당금액", "좌석등급", "할인종류명(전송처)", "결제수단명(전송처)"]].sort_values("장당금액")

,전송사업자명,좌석수,공연일시,예매/취소일시,입장권고유번호,예매/취소금액,할인금액,장당금액,좌석등급,할인종류명(전송처),결제수단명(전송처)
2286851,국립부산국악원,686,2022-10-07 19:30:00,2022-10-05 13:39:00,20221005-133909,348800,523200,8000,"S석(10000), A석(8000)",직?단원?출연자,99
2164766,국립부산국악원,686,2022-09-04 17:00:00,2022-08-17 13:53:00,20220817-135302,240000,240000,8000,"S석(10000), A석(8000)",경로,99
1716477,국립부산국악원,686,2022-05-21 15:00:00,2022-05-21 09:29:00,20220521-092918,280000,280000,8000,"S석(10000), A석(8000)",기업체?기관 할인,99
2294001,국립부산국악원,686,2022-10-08 15:00:00,2022-10-05 13:52:00,20221005-135228,350400,525600,8000,"S석(10000), A석(8000)",직?단원?출연자,99
702155,국립부산국악원,686,2020-07-10 19:30:00,2020-07-10 19:18:00,20200710-191813,221600,90400,8000,"S석(10000), A석(8000)",24세 이하 청소년 및 대학생,99
...,...,...,...,...,...,...,...,...,...,...,...
535472,국립부산국악원,686,2019-12-20 19:30:00,2019-11-28 14:37:00,20191128-143737,4720000,0,4720000,전석(20000),None,99
1367767,티켓링크,455,2021-12-10 19:30:00,2021-12-10 14:09:00,1426356353_1,6600000,0,6600000,전석(30000),None,신용카드 수기결제
102769,예스24,978,2019-08-16 15:00:00,2019-07-10 09:06:00,721126122800091,10000000,0,10000000,"양일권(49500), 일일권(27500)",기타,현금
102768,예스24,978,2019-08-16 15:00:00,2019-07-10 09:06:00,721126122800090,10000000,0,10000000,"양일권(49500), 일일권(27500)",기타,현금


In [67]:
temp = df[df["예매/취소금액"]>200000][["전송사업자명", "좌석수", "공연일시", "예매/취소일시", "입장권고유번호", "예매/취소금액", "할인금액", "장당금액", "좌석등급", "할인종류명(전송처)", "결제수단명(전송처)"]]
temp = temp[(temp["결제수단명(전송처)"]=="현금")|(temp["결제수단명(전송처)"]=="99")|(temp["할인종류명(전송처)"]=="기타")|(temp["할인종류명(전송처)"]==None)]
temp

,전송사업자명,좌석수,공연일시,예매/취소일시,입장권고유번호,예매/취소금액,할인금액,장당금액,좌석등급,할인종류명(전송처),결제수단명(전송처)
102768,예스24,978,2019-08-16 15:00:00,2019-07-10 09:06:00,721126122800090,10000000,0,10000000,"양일권(49500), 일일권(27500)",기타,현금
102769,예스24,978,2019-08-16 15:00:00,2019-07-10 09:06:00,721126122800091,10000000,0,10000000,"양일권(49500), 일일권(27500)",기타,현금
277505,국립부산국악원,686,2019-10-12 17:00:00,2019-09-26 14:40:00,20190926-144012,350000,350000,10000,"R석(20000), S석(10000)",조기예매할인 (8.12.(월)부터~8.31.(토)까지),99
277807,국립부산국악원,686,2019-10-12 17:00:00,2019-10-08 13:36:00,20191008-133632,300000,300000,10000,"R석(20000), S석(10000)",조기예매할인 (8.12.(월)부터~8.31.(토)까지),99
525758,국립부산국악원,686,2019-12-18 19:30:00,2019-12-04 13:46:00,20191204-134633,17200000,0,70000,"S석(100000), A석(70000), B석(30000)",None,99
526434,국립부산국악원,686,2019-12-18 19:30:00,2019-12-17 10:01:00,20191217-100139,2240000,0,70000,"S석(100000), A석(70000), B석(30000)",None,99
526435,국립부산국악원,686,2019-12-18 19:30:00,2019-12-17 10:02:00,20191217-100229,3200000,0,70000,"S석(100000), A석(70000), B석(30000)",None,99
535472,국립부산국악원,686,2019-12-20 19:30:00,2019-11-28 14:37:00,20191128-143737,4720000,0,4720000,전석(20000),None,99
538281,국립부산국악원,686,2019-12-20 19:30:00,2019-12-17 13:52:00,20191217-135241,1573333,66667,1640000,전석(20000),None,99
553025,국립부산국악원,686,2019-12-21 18:00:00,2019-12-18 13:57:00,20191218-135731,2170000,0,10000,"S석(30000), A석(10000)",None,99


In [68]:
# 할인종류와 결제수단이 이상한 애들만 제거 - 명확하지 않은거
idx = [102755,  102756,  523219,  523895,  523896,  532933,  535742, 550482,  550483,  550484, 1864641, 1867821]
df = df.drop(idx, axis=0)
df = df.reset_index(drop=True)

## 할인금액

In [69]:
# 할인
7.000000e+05

700000.0

In [70]:
# 할인금액 700000만원 문제 없어보임.
df[df["할인금액"]==700000][["좌석수", "공연일시", "예매/취소일시", "입장권고유번호", "예매/취소금액", "할인금액", "장당금액", "좌석등급", "할인종류명(전송처)", "결제수단명(전송처)"]]

,좌석수,공연일시,예매/취소일시,입장권고유번호,예매/취소금액,할인금액,장당금액,좌석등급,할인종류명(전송처),결제수단명(전송처)
1976410,686,2022-07-23 17:00:00,2022-07-14 15:45:00,20220714-154559,700000,700000,10000,"S석(20000), A석(10000)",출연자,99


In [71]:
# 그 외 문제없어 보임.
df[df["할인금액"]>=200000][["좌석수", "공연일시", "예매/취소일시", "입장권고유번호", "예매/취소금액", "할인금액", "장당금액", "좌석등급", "할인종류명(전송처)", "결제수단명(전송처)"]]

,좌석수,공연일시,예매/취소일시,입장권고유번호,예매/취소금액,할인금액,장당금액,좌석등급,할인종류명(전송처),결제수단명(전송처)
130446,686,2019-08-24 17:00:00,2019-07-23 14:14:00,20190723-141437,200000,200000,8000,"S석(10000), A석(8000)",[창극<지리산>] 조기 예매 할인,99
277503,686,2019-10-12 17:00:00,2019-09-26 14:40:00,20190926-144012,350000,350000,10000,"R석(20000), S석(10000)",조기예매할인 (8.12.(월)부터~8.31.(토)까지),99
277805,686,2019-10-12 17:00:00,2019-10-08 13:36:00,20191008-133632,300000,300000,10000,"R석(20000), S석(10000)",조기예매할인 (8.12.(월)부터~8.31.(토)까지),99
535465,686,2019-12-20 19:30:00,2019-11-28 14:34:00,20191128-143453,0,200000,0,전석(20000),None,99
812605,686,2020-11-22 15:00:00,2020-11-22 10:22:00,20201122-102256,240000,240000,10000,"S석(20000), A석(10000)",부산시민 50%,99
1030116,686,2021-07-14 19:30:00,2021-07-02 16:36:00,20210702-163605,931000,399000,10000,"S석(20000), A석(10000)",20인 이상 단체 30% (전화예매),99
1072275,686,2021-08-07 17:00:00,2021-07-27 10:53:00,20210727-105315,200000,200000,10000,"S석(20000), A석(10000)",출연자,99
1072279,686,2021-08-07 17:00:00,2021-07-31 18:25:00,20210731-182509,220000,220000,10000,"S석(20000), A석(10000)",출연자,99
1713470,686,2022-05-20 19:30:00,2022-05-20 10:30:00,20220520-103007,200000,200000,8000,"S석(10000), A석(8000)",기업체?기관 할인,99
1716467,686,2022-05-21 15:00:00,2022-05-21 09:29:00,20220521-092918,280000,280000,8000,"S석(10000), A석(8000)",기업체?기관 할인,99


## 장당금액

In [72]:
# 장당
5.260000e+07

52600000.0

In [73]:
# 위에는 예매에서 봤던데이터

In [74]:
df[df["장당금액"]>=200000][["좌석수", "공연일시", "예매/취소일시", "입장권고유번호", "예매/취소금액", "할인금액", "장당금액", "좌석등급", "할인종류명(전송처)", "결제수단명(전송처)"]]

,좌석수,공연일시,예매/취소일시,입장권고유번호,예매/취소금액,할인금액,장당금액,좌석등급,할인종류명(전송처),결제수단명(전송처)
81340,640,2019-08-03 16:00:00,2019-07-04 22:06:00,5466,216750,0,216750,"R석(60000), S석(40000)",None,카드
102766,978,2019-08-16 15:00:00,2019-07-10 09:06:00,721126122800090,10000000,0,10000000,"양일권(49500), 일일권(27500)",기타,현금
102767,978,2019-08-16 15:00:00,2019-07-10 09:06:00,721126122800091,10000000,0,10000000,"양일권(49500), 일일권(27500)",기타,현금
535466,686,2019-12-20 19:30:00,2019-11-28 14:37:00,20191128-143737,4720000,0,4720000,전석(20000),None,99
538274,686,2019-12-20 19:30:00,2019-12-17 13:52:00,20191217-135241,1573333,66667,1640000,전석(20000),None,99
...,...,...,...,...,...,...,...,...,...,...
1956850,640,2022-07-20 19:30:00,2022-06-07 14:39:00,T1955460053,287000,0,287000,"R석(80000), S석(60000)",None,신용카드
1956913,640,2022-07-20 19:30:00,2022-06-23 17:48:00,T1969226560,287000,0,287000,"R석(80000), S석(60000)",None,신용카드
1956914,640,2022-07-20 19:30:00,2022-06-23 17:48:00,T1969226561,287000,0,287000,"R석(80000), S석(60000)",None,신용카드
1957098,640,2022-07-20 19:30:00,2022-06-29 20:04:00,T1973793190,287000,0,287000,"R석(80000), S석(60000)",None,신용카드


In [75]:
# 할인종류가 명확해서 큰 문제 없음
temp = df[df["장당금액"]>200000][["전송사업자명", "좌석수", "공연일시", "예매/취소일시", "입장권고유번호", "예매/취소금액", "할인금액", "장당금액", "좌석등급", "할인종류명(전송처)", "결제수단명(전송처)"]]
temp = temp[(temp["결제수단명(전송처)"]=="현금")|(temp["결제수단명(전송처)"]=="99")|(temp["할인종류명(전송처)"]=="기타")|(temp["할인종류명(전송처)"]==None)]
temp

,전송사업자명,좌석수,공연일시,예매/취소일시,입장권고유번호,예매/취소금액,할인금액,장당금액,좌석등급,할인종류명(전송처),결제수단명(전송처)
102766,예스24,978,2019-08-16 15:00:00,2019-07-10 09:06:00,721126122800090,10000000,0,10000000,"양일권(49500), 일일권(27500)",기타,현금
102767,예스24,978,2019-08-16 15:00:00,2019-07-10 09:06:00,721126122800091,10000000,0,10000000,"양일권(49500), 일일권(27500)",기타,현금
535466,국립부산국악원,686,2019-12-20 19:30:00,2019-11-28 14:37:00,20191128-143737,4720000,0,4720000,전석(20000),None,99
538274,국립부산국악원,686,2019-12-20 19:30:00,2019-12-17 13:52:00,20191217-135241,1573333,66667,1640000,전석(20000),None,99
1680811,네이버N예약,994,2022-05-13 19:30:00,2022-04-24 18:45:00,264227887_0,286000,0,286000,"R석(143000), S석(121000)",기타,카드
1680829,네이버N예약,994,2022-05-13 19:30:00,2022-04-25 13:46:00,264495850_0,286000,0,286000,"R석(143000), S석(121000)",기타,카드
1680833,네이버N예약,994,2022-05-13 19:30:00,2022-04-25 22:57:00,264722870_0,286000,0,286000,"R석(143000), S석(121000)",기타,카드
1680954,네이버N예약,994,2022-05-13 19:30:00,2022-04-29 20:28:00,266577686_0,286000,0,286000,"R석(143000), S석(121000)",기타,카드
1680965,네이버N예약,994,2022-05-13 19:30:00,2022-05-01 00:04:00,267096938_0,286000,0,286000,"R석(143000), S석(121000)",기타,카드
1680975,네이버N예약,994,2022-05-13 19:30:00,2022-05-01 20:00:00,267429381_0,286000,0,286000,"R석(143000), S석(121000)",기타,카드


In [76]:
# df.to_parquet("After_Medium_Theater.parquet")

# Make data

## 군집분석 데이터(+파생변수)

In [ ]:
df = pd.read_parquet("data/After_Medium_Theater.parquet")

In [80]:
# 평균좌석점유율 구하기
temp = df[(df["공연일시"]>="2019-07-01")&(df["공연일시"]<"2020-04-01")]
gb = temp.groupby(["공연시설_공연_코드", "공연일시", "좌석수"]).sum()
gb = gb.reset_index()
gb["회차별좌석점유율"] = gb["예매/취소구분"] / gb["좌석수"]
gb = gb.groupby(["공연시설_공연_코드", "좌석수"]).mean().reset_index()
gb = gb[["공연시설_공연_코드", "회차별좌석점유율"]]

In [81]:
df["평균좌석점유율"] = ""

def MakeAverageSeat (temp, idx, gb):
    for i in idx:
        code = df.iloc[i]["공연시설_공연_코드"]
        temp.loc[i, "평균좌석점유율"] = gb[gb["공연시설_공연_코드"]==code]["회차별좌석점유율"].values[0]
    return temp

idx = temp.index
temp = MakeAverageSeat(temp, idx, gb)

C:\Users\lsj\AppData\Local\Temp\ipykernel_13740\3648638926.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.loc[i, "평균좌석점유율"] = gb[gb["공연시설_공연_코드"]==code]["회차별좌석점유율"].values[0]


In [82]:
temp = temp.astype({'평균좌석점유율':'float64'})

In [83]:
temp = temp.groupby(["공연시설코드", "주소"]).mean()

In [84]:
temp = temp.reset_index()

In [85]:
# temp.to_parquet("Medium_Theater_Cluster_adress.parquet")

## 군집분석 공연장 수정(전체매출시각화)

In [87]:
df = pd.read_parquet("data/After_Medium_Theater.parquet")

In [ ]:
cluster = pd.read_parquet("data/Medium_Theater_Cluster_adress.parquet")

In [90]:
idx = cluster[cluster["무료여부"]==1].index

cluster = cluster.drop(idx, axis=0)
cluster = cluster.reset_index(drop=True)

In [91]:
len(cluster["공연시설코드"].unique())

177

In [92]:
len(df["공연시설코드"].unique())

222

In [93]:
code = cluster["공연시설코드"].tolist()

In [94]:
len(df[df["공연시설코드"].apply(lambda x : x in code)])

2603642

In [95]:
len(df)

2785422

In [96]:
df = df[df["공연시설코드"].apply(lambda x : x in code)]

In [97]:
len(df["공연시설코드"].unique())

177

In [98]:
temp = df.groupby(["공연시설코드", "년도_월"]).mean()

In [99]:
temp = temp.reset_index()[["공연시설코드", "년도_월", "예매/취소금액"]]

In [100]:
idx = temp["공연시설코드"].unique()
len(idx)

177

In [101]:
# 충격지점 이후 0이 아닌 데이터가 2개이상인 극장만 선정
res = []
cnt = 1
for i in range(len(idx)):
    draw_df = temp[temp["공연시설코드"]==idx[i]]
    data_after_time_point = draw_df[(draw_df["년도_월"] >= 202004)&(draw_df["년도_월"] <= 202212)][["예매/취소금액"]]
    data_after_time_point_len = len(data_after_time_point)

    if (data_after_time_point_len <= 1):
        res.append(idx[i])
    else:
        for j in range(data_after_time_point_len):
            if data_after_time_point.iloc[j].values[0] != 0:
                cnt = cnt+1
        if cnt == 1:
            res.append(idx[i])
    cnt=1

In [102]:
idx = cluster[cluster["공연시설코드"].apply(lambda x : x in res)].index
idx

Int64Index([  7,  31,  37,  41,  42,  51,  52,  65,  66,  71,  78,  89,  92,
            100, 107, 108, 110, 124, 134, 138, 139, 146, 150, 153, 157, 159,
            161, 162, 163, 165, 166, 168, 169, 170, 171, 173, 174, 175, 176],
           dtype='int64')

In [103]:
cluster = cluster.drop(idx, axis=0)
cluster = cluster.reset_index(drop=True)

In [104]:
idx = temp[temp["공연시설코드"].apply(lambda x : x in res)].index
idx

Int64Index([ 169,  170,  713,  714,  715,  716,  717,  718,  719,  861,
            ...
            2861, 2862, 2863, 2864, 2865, 2874, 2875, 2876, 2877, 2878],
           dtype='int64', length=102)

In [105]:
temp = temp.drop(idx, axis=0)
temp = temp.reset_index(drop=True)

In [106]:
idx = temp["공연시설코드"].unique()
len(idx)

138

In [107]:
idx = cluster["공연시설코드"].unique()
len(idx)

138

In [108]:
# cluster.to_parquet("Medium_Theater_remove_cluster.parquet")

In [109]:
# cluster.to_csv("Medium_Theater_remove_cluster.csv")

## 예상매출 학습 데이터

In [ ]:
df = pd.read_parquet("data/After_Medium_Theater.parquet")

In [123]:
temp = df[(df["공연일시"]>="2019-07-01")&(df["공연일시"]<"2020-04-01")]
temp = temp.groupby(["공연시설코드", "년도_월", "좌석수", "년도_월_일"]).sum()

In [124]:
temp = temp.reset_index()[["공연시설코드", "년도_월", "년도_월_일", "좌석수", "예매/취소금액", "예매/취소구분"]]

In [ ]:
cluster = pd.read_parquet("data/Medium_Theater_remove_cluster.parquet")

In [126]:
len(cluster["공연시설코드"].unique())

138

In [127]:
len(temp["공연시설코드"].unique())

186

In [128]:
code = cluster["공연시설코드"].tolist()

In [129]:
len(temp[temp["공연시설코드"].apply(lambda x : x in code)])

2578

In [130]:
temp = temp[temp["공연시설코드"].apply(lambda x : x in code)]

In [131]:
len(temp["공연시설코드"].unique())

138

In [132]:
idx = temp["공연시설코드"].unique()
len(idx)

138

In [121]:
# temp.to_csv("Medium_Theater_estimate.csv", index=False)

## 회복탄력성 데이터

In [ ]:
# 군집분석 공연시설과 기간이 다르기 때문에 군집분석 데이터 기준으로 공연시설코드 통일시켜주기

In [ ]:
df = pd.read_parquet("data/After_Medium_Theater.parquet")

In [134]:
temp = df[(df["공연일시"] >= "2020-04-01")&(df["공연일시"] < "2023-01-01")]
temp = temp.groupby(["공연시설코드", "년도_월"]).sum()
temp

좌석수  예매/취소구분   예매/취소금액  예매/취소방식코드  결제수단코드    할인금액  할인종류코드  \
공연시설코드   년도_월                                                                   
FC222006 202208   22365       45    350000         87     682       0    4455   
         202209   23359       47    221000         84    2044       0    4653   
FC222007 202103    8256       16    234000         22     526   30000    1584   
         202105  243036      471   9572500        853   13501       0   46629   
         202106     516        1     10000          1       2       0      99   
...                 ...      ...       ...        ...     ...     ...     ...   
FC225265 202210  238640      628  11609200       2414   14518       0   62172   
         202211  394440     1038  20810879      12780   34765       0  102762   
         202212  145920      384   6912000        721   22607       0   38016   
FC225327 202209   60000      200         0      16373     204       0   19800   
FC225373 202212  279900      933  31158400       1670   60810  437600   92367   

                     장당금액        년도_월_일       년도      월      일  예매건수  취소건수  \
공연시설코드   년도_월                                                                
FC222006 202208    350000  9.099365e+08    90990    360    526    45     0   
         202209    221000  9.503837e+08    95034    423   1382    47     0   
FC222007 202103    264000  3.233653e+08    32336     48    480    16     0   
         202105   9572500  9.519150e+09   951891   2355   4029   471     0   
         202106     10000  2.021062e+07     2021      6     24     1     0   
...                   ...           ...      ...    ...    ...   ...   ...   
FC225265 202210  11609200  1.269880e+10  1269816   6280   7278   628     0   
         202211  20810879  2.098952e+10  2098836  11418  18455  1038     0   
         202212   6912000  7.764950e+09   776448   4608   9216   384     0   
FC225327 202209         0  4.044181e+09   404400   1800    600   200     0   
FC225373 202212  31596000  1.886640e+10  1886526  11196  21678   933     0   

                 무료여부  
공연시설코드   년도_월          
FC222006 202208    90  
         202209    94  
FC222007 202103    32  
         202105   942  
         202106     2  
...               ...  
FC225265 202210  1256  
         202211  2076  
         202212   768  
FC225327 202209   200  
FC225373 202212  1866  

[2390 rows x 15 columns]

In [135]:
temp = temp.reset_index()
temp = temp[["공연시설코드", "년도_월", "예매/취소금액"]]

In [136]:
len(temp["공연시설코드"].unique())

193

In [ ]:
cluster = pd.read_parquet("data/Medium_Theater_remove_cluster.parquet")

In [138]:
len(cluster["공연시설코드"].unique())

138

In [139]:
len(temp["공연시설코드"].unique())

193

In [140]:
code = cluster["공연시설코드"].tolist()

In [141]:
len(temp[temp["공연시설코드"].apply(lambda x : x in code)])

2119

In [142]:
temp = temp[temp["공연시설코드"].apply(lambda x : x in code)]

In [143]:
len(temp["공연시설코드"].unique())

138

In [144]:
temp = temp.reset_index(drop=True)

In [ ]:
# temp.to_csv("Medium_Theater_resilience.csv", index=False)